<a href="https://colab.research.google.com/github/sungmino/Learn-standard/blob/master/SAV_smn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!nvidia-smi

Sat May 15 03:36:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
!pip install fairseq

In [ ]:
!pip install transformers

In [ ]:
!pip install fastBPE

In [ ]:
!pip install vncorenlp

In [9]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/drive/MyDrive/SA_Projects/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--bpe-codes',
                        default="/content/drive/MyDrive/SA_Projects/PhoBERT_base_transformers/bpe.codes",
                        required=False,
                        type=str,
                        help='path to fastBPE BPE'
                        )
args, unknown = parser.parse_known_args()
bpe = fastBPE(args)
    # Load the dictionary
vocab = Dictionary()
vocab.add_from_file("/content/drive/MyDrive/SA_Projects/PhoBERT_base_transformers/dict.txt")

In [10]:
bpe.encode("Hôm_nay trời nóng quá nên tôi ở nhà viết Viblo!")

'Hôm_nay trời nóng quá nên tôi ở nhà viết Vi@@ blo@@ !'

In [11]:
import re
import json
from tqdm import tqdm_notebook

train_path = '/content/drive/MyDrive/SA_Projects/data/train/train.json'
test_path = '/content/drive/MyDrive/SA_Projects/data/test/test.json'

def load_data_format(filename1):
    with open(filename1, 'r') as file:
        contents = json.load(file)
        file.close()
    return contents

train_id, train_text, train_label = [], [], []
test_id, test_text = [], []

train_data = load_data_format(train_path)
train_data = train_data["reviews"]
for k in tqdm_notebook(train_data):
    train_id.append(k["id"])
    text = k["comment"]
    text = rdrsegmenter.tokenize(text)
    text = ' '.join([' '.join(x) for x in text])
    train_text.append(text)
    train_label.append(k["label"])

test_data = load_data_format(test_path)
test_data = test_data["reviews"]
for n in tqdm_notebook(test_data):
    test_id.append(n["id"])
    texts = n["comment"]
    texts = rdrsegmenter.tokenize(texts)
    texts = ' '.join([' '.join(x) for x in texts])
    test_text.append(texts)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [12]:
from sklearn.model_selection import train_test_split

train_sents, val_sents, train_labels, val_labels = train_test_split(train_text, train_label, test_size=0.1)

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_LEN = 150
train_ids = []
for tx in train_sents:
    subwords = '<s> ' + bpe.encode(tx) + ' </s>'
    encoded_tx = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    train_ids.append(encoded_tx)
val_ids = []
for tx in val_sents:
    subwords = '<s> ' + bpe.encode(tx) + ' </s>'
    encoded_tx = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    val_ids.append(encoded_tx)

# tao list id của các subword có trong từ điển
train_ids = pad_sequences(train_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
val_ids = pad_sequences(val_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

In [14]:
 # tao mask, mask cho biet cac value nao da duoc padding
train_masks = []
for tx in train_ids:
    mask = [int(token_id > 0) for token_id in tx]
    train_masks.append(mask)
val_masks = []
for tx in val_ids:
    mask = [int(token_id > 0) for token_id in tx]
    val_masks.append(mask)

In [15]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch

# tao dataloader
train_inputs = torch.tensor(train_ids)
val_inputs = torch.tensor(val_ids)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=32)

In [16]:
train_dataloader

In [17]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

config = RobertaConfig.from_pretrained(
        "/content/drive/MyDrive/SA_Projects/PhoBERT_base_transformers/config.json", from_tf=False, num_labels=2, output_hidden_states=False,
    )

BERT_SA = RobertaForSequenceClassification.from_pretrained(
      "/content/drive/MyDrive/SA_Projects/PhoBERT_base_transformers/model.bin",
      config=config
    )

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at /content/drive/MyDrive/SA_Projects/PhoBERT_base_transformers/model.bin were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassific

In [18]:
BERT_SA.cuda()
print('OK DONE...')

OK DONE...


In [20]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    F1_score = f1_score(pred_flat, labels_flat, average='macro')
    
    return accuracy_score(pred_flat, labels_flat), F1_score

In [21]:
import random
from tqdm import tqdm_notebook
device = 'cuda'
epochs = 4
from collections import defaultdict


param_optimizer = list(BERT_SA.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5, correct_bias=False)

history = defaultdict(list)
best_accuracy = 0

for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    total_loss = 0
    BERT_SA.train()
    train_accuracy = 0
    nb_train_steps = 0
    train_f1 = 0
    if torch.cuda.is_available():
          BERT_SA.to(device)
    for step, batch in tqdm_notebook(enumerate(train_dataloader)):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        BERT_SA.zero_grad()
        outputs = BERT_SA(b_input_ids, 
            token_type_ids=None, 
            attention_mask=b_input_mask, 
            labels=b_labels)
        loss = outputs[0]
        total_loss += loss.item()
        
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_train_accuracy, tmp_train_f1 = flat_accuracy(logits, label_ids)
        train_accuracy += tmp_train_accuracy
        train_f1 += tmp_train_f1
        nb_train_steps += 1
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(BERT_SA.parameters(), 1.0)
        optimizer.step()
        
    avg_train_loss = total_loss / len(train_dataloader)
    print(" Accuracy: {0:.4f}".format(train_accuracy/nb_train_steps))
    print(" F1 score: {0:.4f}".format(train_f1/nb_train_steps))
    print(" Average training loss: {0:.4f}".format(avg_train_loss))
    train_acc = train_accuracy/nb_train_steps
    train_f1c = train_f1/nb_train_steps
    train_loss = avg_train_loss

    print("Running Validation...")
    BERT_SA.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    eval_f1 = 0
    for batch in tqdm_notebook(val_dataloader):

        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = BERT_SA(b_input_ids, 
            token_type_ids=None, 
            attention_mask=b_input_mask)
            logits = outputs[0]
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            tmp_eval_accuracy, tmp_eval_f1 = flat_accuracy(logits, label_ids)

            eval_accuracy += tmp_eval_accuracy
            eval_f1 += tmp_eval_f1
            nb_eval_steps += 1
    print(" Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
    print(" F1 score: {0:.4f}".format(eval_f1/nb_eval_steps))

    val_acc = eval_accuracy/nb_eval_steps
    val_f1c = eval_f1/nb_eval_steps

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['train_f1'].append(train_f1c)
    history['val_acc'].append(val_acc)
    history['val_f1'].append(val_f1c)

    if (val_acc > best_accuracy):
      torch.save(BERT_SA.state_dict(), '/content/drive/MyDrive/SA_Projects/best_model_state.bin')
      best_accuracy = val_acc
print("Training complete!") 

======== Epoch 1 / 4 ========
Training...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



 Accuracy: 0.9270
 F1 score: 0.9243
 Average training loss: 0.1926
Running Validation...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



 Accuracy: 0.9443
 F1 score: 0.9427
======== Epoch 2 / 4 ========
Training...


...


 Accuracy: 0.9529
 F1 score: 0.9515
======== Epoch 3 / 4 ========
Training...



 Accuracy: 0.9681
 F1 score: 0.9670
 Average training loss: 0.1055
Running Validation...



 Accuracy: 0.9559
 F1 score: 0.9546
======== Epoch 4 / 4 ========
Training...



 Accuracy: 0.9746
 F1 score: 0.9738
 Average training loss: 0.0873
Running Validation...



 Accuracy: 0.9606
 F1 score: 0.9595
Training complete!
